# Initilialize the Environment

In [ ]:
import os 
os.chdir('..')
import Env_0 as robotEnv

import mujoco 
from scipy.spatial.transform import Rotation
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
env = robotEnv.robotEnv({'ROS_ID':0})
_ = env.reset( )

In [ ]:
dataset_folder = "datasets/test"

# Define useful functions

In [ ]:
def compute_camera_matrix(renderer, data,model):
  renderer.update_scene(data,camera="top_down")
  pos = np.mean([camera.pos for camera in renderer.scene.camera], axis=0)
  z = -np.mean([camera.forward for camera in renderer.scene.camera], axis=0)
  y = np.mean([camera.up for camera in renderer.scene.camera], axis=0)
  rot = np.vstack((np.cross(y, z), y, z))
  fov = model.cam('top_down').fovy[0]

  # Translation matrix (4x4).
  translation = np.eye(4)
  translation[0:3, 3] = -pos

  # Rotation matrix (4x4).
  rotation = np.eye(4)
  rotation[0:3, 0:3] = rot

  # Focal transformation matrix (3x4).
  focal_scaling = (1./np.tan(np.deg2rad(fov)/2)) * renderer.height / 2.0
  focal = np.diag([-focal_scaling, focal_scaling, 1.0, 0])[0:3, :]

  # Image matrix (3x3).
  image = np.eye(3)
  image[0, 2] = (renderer.width - 1) / 2.0
  image[1, 2] = (renderer.height - 1) / 2.0
  return image @ focal @ rotation @ translation

In [ ]:
def word2pixel(xyz_global,cam_matrix):
    # Camera matrices multiply homogenous [x, y, z, 1] vectors.
    corners_homogeneous = np.ones((4, xyz_global.shape[1]), dtype=float)
    corners_homogeneous[:3, :] = xyz_global
    
    # Get the camera matrix.
    xs, ys, s = cam_matrix @ corners_homogeneous
    
    # x and y are in the pixel coordinate system.
    x = xs / s
    y = ys / s
    return x,y

In [ ]:
def normalizeAngle(angle):
    if(angle>np.pi):angle -=np.pi
    elif(angle<0):angle += np.pi
    return angle

# Generate the dataset


In [ ]:
cam_matrix = compute_camera_matrix(env.simulator.renderer, env.simulator.data,env.simulator.model)

In [ ]:
# Create dataset label file
f = open(f"{dataset_folder}/labels.csv", "a+")
f.write("img,x1,y1,rot1,valrot1,validity1,x2,y2,rot2,valrot2,validity2,x3,y3,rot3,valrot3,validity3\n")

In [ ]:
for j in range(8000):
    
    if(j%100==0): print(j)

    env.reset()
    obs = env.simulator.get_state()
    initialObjPos=[]
    
    for index in range(env.simulator.constants["NUMBER_OF_OBJECTS"]):
        # get the position of the obj from the site in the right position (+1 due to the target site)
        initialObjPos.extend(env.simulator.data.site(1+index).xpos.copy())
        #get the rotation of the obj
        initialObjPos.extend(env.simulator.data.qpos[0+7*index+3:7*(index+1)].copy()) 
    
    f.write(f"img{j}.png")
    
    for index in range(env.simulator.constants["NUMBER_OF_OBJECTS"]):

        obj_position= initialObjPos[0+7*index:3+7*index]
        rot = Rotation.from_quat(initialObjPos[3+7*index:7+7*index]).as_euler('xyz')
        
        #trasform the position of the object in pixel coordinates
        pixelCoord = word2pixel(np.array([obj_position[0],obj_position[1],0.20921954]).reshape(3,1),cam_matrix)

        #write the label in the file
        f.write(f",{float(pixelCoord[0])},{float(pixelCoord[1])},{rot[0]},{rot[2]},{(rot[2]<0.7 and rot[2]>0)}")
    
    f.write("\n")
    Image.fromarray(obs).save(f"{dataset_folder}/img{j}.png")
    
f.close()
